In [1]:
import requests

In [2]:
urls = {
    ".gitattributes": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/.gitattributes?download=true", 
    "README.md": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/README.md?download=true", 
    "config.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json?download=true", 
    "generation_config.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/generation_config.json?download=true", 
    "model-00001-of-00003.safetensors": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/model-00001-of-00003.safetensors?download=true", 
    "model-00002-of-00003.safetensors": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/model-00002-of-00003.safetensors?download=true", 
    "model-00003-of-00003.safetensors": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/model-00003-of-00003.safetensors?download=true", 
    "model.safetensors.index.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/model.safetensors.index.json?download=true", 
    "pytorch_model-00001-of-00003.bin": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00001-of-00003.bin?download=true", 
    "pytorch_model-00002-of-00003.bin": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00002-of-00003.bin?download=true", 
    "pytorch_model-00003-of-00003.bin": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00003-of-00003.bin?download=true", 
    "pytorch_model.bin.index.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model.bin.index.json?download=true", 
    "special_tokens_map.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/special_tokens_map.json?download=true", 
    "tokenizer.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/tokenizer.json?download=true", 
    "tokenizer.model": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/tokenizer.model?download=true", 
    "tokenizer_config.json": "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/tokenizer_config.json?download=true"
}

In [3]:
for name, url in urls.items():
  response = requests.get(url)
  print("*"*40)
  print(response.ok, response.status_code, url, sep="\n")
  if response.ok and response.status_code == 200:
    with open(f"../model/{name}", mode="wb") as file:
      file.write(response.content)
  del response

****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00001-of-00003.bin?download=true
****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00002-of-00003.bin?download=true
****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model-00003-of-00003.bin?download=true
****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/pytorch_model.bin.index.json?download=true
****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/special_tokens_map.json?download=true
****************************************
True
200
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/tokenizer.json?download=true
***********************

In [4]:
# Sample 1

from transformers import AutoTokenizer, MistralForCausalLM

model = MistralForCausalLM.from_pretrained("../model")
tokenizer = AutoTokenizer.from_pretrained("../model")

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# Sample 2

from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cpu" # the device to load the model onto # cuda # cpu

model = AutoModelForCausalLM.from_pretrained("../model")
tokenizer = AutoTokenizer.from_pretrained("../model")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])